# How commute time worsened in Massachussetts?
Recently, I saw [Boston.com reports](https://www.boston.com/news/local-news/2022/03/18/mass-workers-have-the-4th-longest-commute-among-u-s-states-according-to-new-census-data/#:~:text=Massachusetts%20had%20the%20highest%20increase,period%20of%202016%20to%202020) that Massachusetts has one of the worst commute time for workers among US states. I wanted to see how bad it really is in the state and how much it has improved or worsened over the last 10 years.

For this visualization piece, I depend on the 2015 ACS and the 2020 ACS. Thankfully, the [ACS Table B01003](https://censusreporter.org/tables/B08303/) is a robust data on specific commute time periods. In this table, I target 4 columns: population of workers who travel 5-9 minutes, 10-14 minutes, 15-19 minutes and 20 to 24 minutes.

In [ ]:
!pip install geopandas
!pip install cenpy
!pip install jenkspy
!pip install jsonschema==3.2
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
import cenpy as cen
import cenpy.tiger as tiger
import jenkspy
from getpass import getpass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
CENSUS_API_KEY = getpass('Enter your Census API Key: ')

Enter your Census API Key: ··········


## In this table, I target 4 columns: population of workers who travel within the tiime range of 5 to 45 minutes.







In [ ]:

con20 = cen.remote.APIConnection('ACSDT5Y2020',apikey=CENSUS_API_KEY)
con15 = cen.remote.APIConnection('ACSDT5Y2015',apikey=CENSUS_API_KEY)


columns = [
    'B01003_001', # Total Population
    'B08303_003', #5 mins to 9 mins
    'B08303_004', #10 mins to 14 mins
    'B08303_005', #15 mins to 19 mins
    'B08303_006', #20 mins to 24 mins
    'B08303_007', #25 mins to 29 mins
    'B08303_008', #30 mins to 34 mins
    'B08303_009', #35 mins to 39 mins
    'B08303_010', #40 mins to 44 mins
    

]


columns_E = [i+'E' for i in columns]
columns_M = [i+'M' for i in columns]
column_estimates = columns_E + columns_M

# Define the units of geography, as well as a filter for what we are looking for. 
# State codes can be found at https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
# You can also use the Missouri Census Data Center, which has great resources for lookups - https://mcdc.missouri.edu/
# Geographies: zip code tabulation area, school district (elementary), school district (secondary), combined statistical area, congressional district, county, tract, block group
# Filters on the geography: filter to tracts within state and county = {'state':'12', 'county':'073'}
g_unit = 'county'
g_filter = {'state':'25'} #location of geography that you want the data for
# g_filter = {'state':'27', 'county': ''} #this grabs all variables for all counties in all states in the country


commute20 = con20.query(column_estimates, geo_unit=g_unit, geo_filter=g_filter)
commute15 = con15.query(column_estimates, geo_unit=g_unit, geo_filter=g_filter)

In [ ]:
pd.set_option('display.max_rows', 14)
pd.set_option('display.max_columns', 23)


In [ ]:
commute20.sort_values(by='county').head()

,B01003_001E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B01003_001M,B08303_003M,B08303_004M,B08303_005M,B08303_006M,B08303_007M,B08303_008M,B08303_009M,B08303_010M,state,county
0,213505,12039,16159,16749,13013,5728,9769,1913,2004,-555555555,1002,839,1115,1050,613,845,315,323,25,001
4,125927,10436,12119,9195,6624,2749,4834,1224,1528,-555555555,735,822,650,588,319,464,217,279,25,003
5,563301,22503,37501,38709,33284,17200,30790,8874,9619,-555555555,944,1665,1583,1410,1084,1569,805,800,25,005
1,17430,1587,1715,1536,567,118,481,0,49,-555555555,372,338,331,208,76,220,21,75,25,007
6,787038,31599,50450,50321,46234,18789,45530,10550,15979,-555555555,1435,1960,1816,1784,1219,1862,844,951,25,009


In [ ]:
commute15.sort_values(by='county').head()

,B01003_001E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B01003_001M,B08303_003M,B08303_004M,B08303_005M,B08303_006M,B08303_007M,B08303_008M,B08303_009M,B08303_010M,state,county
0,214766,13036,18468,16326,12738,4724,9040,1668,2693,-555555555,918,965,1019,835,491,690,298,426,25,001
1,129288,10867,10791,9684,7267,2898,5001,960,1371,-555555555,891,686,709,583,328,483,188,310,25,003
8,552763,24843,37481,39413,34405,15765,29171,8043,8820,-555555555,1281,1324,1314,1340,813,1044,719,586,25,005
6,17048,1102,1913,1994,939,275,307,67,25,-555555555,374,314,396,242,188,136,83,28,25,007
10,763849,34584,49829,49329,43989,17171,45068,9146,14112,-555555555,1257,1686,1654,1448,988,1405,707,875,25,009


I merge the 2015 table to the 2020 table


In [ ]:

commute = commute15.merge(commute20, on='county')

In [ ]:
commute.head()

,B01003_001E_x,B08303_003E_x,B08303_004E_x,B08303_005E_x,B08303_006E_x,B08303_007E_x,B08303_008E_x,B08303_009E_x,B08303_010E_x,B01003_001M_x,B08303_003M_x,...,B08303_010E_y,B01003_001M_y,B08303_003M_y,B08303_004M_y,B08303_005M_y,B08303_006M_y,B08303_007M_y,B08303_008M_y,B08303_009M_y,B08303_010M_y,state_y
0,214766,13036,18468,16326,12738,4724,9040,1668,2693,-555555555,918,...,2004,-555555555,1002,839,1115,1050,613,845,315,323,25
1,129288,10867,10791,9684,7267,2898,5001,960,1371,-555555555,891,...,1528,-555555555,735,822,650,588,319,464,217,279,25
2,503681,18567,26515,28817,28203,13551,30313,7113,11100,-555555555,953,...,11378,-555555555,1348,1398,1472,1458,992,1626,696,943,25
3,687721,24462,31915,36343,36082,17124,49397,12074,19240,-555555555,1345,...,19822,-555555555,1147,1257,1402,1512,1055,1506,791,1101,25
4,10556,1615,1846,1029,255,33,79,0,11,-555555555,190,...,253,-555555555,263,368,278,161,13,43,7,197,25


In [ ]:
commute = commute[['B01003_001E_x', 'B08303_003E_x', 'B08303_004E_x', 'B08303_005E_x', 'B08303_006E_x', 'B08303_007E_x', 'B08303_008E_x', 'B08303_009E_x', 'B08303_010E_x', 'B01003_001E_y', 'B08303_003E_y', 'B08303_004E_y', 'B08303_005E_y', 'B08303_006E_y', 'B08303_007E_y', 'B08303_008E_y', 'B08303_009E_y', 'B08303_010E_y','state_x', 'county']]
commute.head()

,B01003_001E_x,B08303_003E_x,B08303_004E_x,B08303_005E_x,B08303_006E_x,B08303_007E_x,B08303_008E_x,B08303_009E_x,B08303_010E_x,B01003_001E_y,B08303_003E_y,B08303_004E_y,B08303_005E_y,B08303_006E_y,B08303_007E_y,B08303_008E_y,B08303_009E_y,B08303_010E_y,state_x,county
0,214766,13036,18468,16326,12738,4724,9040,1668,2693,213505,12039,16159,16749,13013,5728,9769,1913,2004,25,001
1,129288,10867,10791,9684,7267,2898,5001,960,1371,125927,10436,12119,9195,6624,2749,4834,1224,1528,25,003
2,503681,18567,26515,28817,28203,13551,30313,7113,11100,518597,20969,28434,27885,28466,12536,30236,8693,11378,25,023
3,687721,24462,31915,36343,36082,17124,49397,12074,19240,703740,20946,29404,33301,35653,17380,46313,12544,19822,25,021
4,10556,1615,1846,1029,255,33,79,0,11,11212,1363,1794,1115,338,8,65,5,253,25,019


Renaming the columns makes it easy to identify them...

In [ ]:
commute = commute.rename(columns={'B01003_001E_x': 'totPop15', 'B08303_003E_x': '5to9mins15', 'B08303_004E_x': '10to14mins15', 'B08303_005E_x': '15to19mins15', 'B08303_006E_x': '20to24mins15', 'B08303_007E_x': '25to29mins15', 'B08303_008E_x': '30to34mins15', 'B08303_009E_x': '35to39mins15', 'B08303_010E_x': '40to44mins15'})
commute = commute.rename(columns={'B01003_001E_y': 'totPop20', 'B08303_003E_y': '5to9mins20', 'B08303_004E_y': '10to14mins20', 'B08303_005E_y': '15to19mins20', 'B08303_006E_y': '20to24mins20', 'B08303_007E_y': '25to29mins20', 'B08303_008E_y': '30to34mins20', 'B08303_009E_y': '35to39mins20', 'B08303_010E_y': '40to44mins20'})


In [ ]:
commute.head()

,totPop15,5to9mins15,10to14mins15,15to19mins15,20to24mins15,25to29mins15,30to34mins15,35to39mins15,40to44mins15,totPop20,5to9mins20,10to14mins20,15to19mins20,20to24mins20,25to29mins20,30to34mins20,35to39mins20,40to44mins20,state_x,county
0,214766,13036,18468,16326,12738,4724,9040,1668,2693,213505,12039,16159,16749,13013,5728,9769,1913,2004,25,001
1,129288,10867,10791,9684,7267,2898,5001,960,1371,125927,10436,12119,9195,6624,2749,4834,1224,1528,25,003
2,503681,18567,26515,28817,28203,13551,30313,7113,11100,518597,20969,28434,27885,28466,12536,30236,8693,11378,25,023
3,687721,24462,31915,36343,36082,17124,49397,12074,19240,703740,20946,29404,33301,35653,17380,46313,12544,19822,25,021
4,10556,1615,1846,1029,255,33,79,0,11,11212,1363,1794,1115,338,8,65,5,253,25,019


I convert each value to string in order to make it possible to do calculations with them

In [ ]:
commute['totPop15'] = commute['totPop15'].astype(int)
commute['5to9mins15'] = commute['5to9mins15'].astype(int)
commute['10to14mins15'] = commute['10to14mins15'].astype(int)
commute['15to19mins15'] = commute['15to19mins15'].astype(int)
commute['20to24mins15'] = commute['20to24mins15'].astype(int)
commute['25to29mins15'] = commute['25to29mins15'].astype(int)
commute['30to34mins15'] = commute['30to34mins15'].astype(int)
commute['35to39mins15'] = commute['35to39mins15'].astype(int)
commute['40to44mins15'] = commute['40to44mins15'].astype(int)
commute['totPop20'] = commute['totPop20'].astype(int)
commute['5to9mins20'] = commute['5to9mins20'].astype(int)
commute['10to14mins20'] = commute['10to14mins20'].astype(int)
commute['15to19mins20'] = commute['15to19mins20'].astype(int)
commute['20to24mins20'] = commute['20to24mins20'].astype(int)
commute['25to29mins20'] = commute['25to29mins20'].astype(int)
commute['30to34mins20'] = commute['30to34mins20'].astype(int)
commute['35to39mins20'] = commute['35to39mins20'].astype(int)
commute['40to44mins20'] = commute['40to44mins20'].astype(int)

...then add columns together to limit the time frame I would be mapping with. Eight columns become 4 columns

In [ ]:
#Combine the columns 
commute['5to14mins15'] = commute['5to9mins15'] + commute['10to14mins15'] 
commute['15to24mins15'] = commute['15to19mins15'] + commute['20to24mins15']
commute['25to34mins15'] = commute['25to29mins15'] + commute['30to34mins15']
commute['35to44mins15'] = commute['35to39mins15'] + commute['40to44mins15']
commute['5to14mins20'] = commute['5to9mins20'] + commute['10to14mins20'] 
commute['15to24mins20'] = commute['15to19mins20'] + commute['20to24mins20']
commute['25to34mins20'] = commute['25to29mins20'] + commute['30to34mins20']
commute['35to44mins20'] = commute['35to39mins20'] + commute['40to44mins20']

In [ ]:
#turn them to percentages
commute['5to14mins15'] = commute['5to14mins15'] / commute['totPop15'] * 100
commute['15to24mins15'] = commute['15to24mins15'] / commute['totPop15'] * 100
commute['25to34mins15'] = commute['25to34mins15'] / commute['totPop15'] * 100
commute['35to44mins15'] = commute['35to44mins15'] / commute['totPop15'] * 100
commute['5to14mins20'] = commute['5to14mins20'] / commute['totPop20'] * 100
commute['15to24mins20'] = commute['15to24mins20'] / commute['totPop20'] * 100
commute['25to34mins20'] = commute['25to34mins20'] / commute['totPop20'] * 100
commute['35to44mins20'] = commute['35to44mins20'] / commute['totPop20'] * 100

In [ ]:
commute.head()

,totPop15,5to9mins15,10to14mins15,15to19mins15,20to24mins15,25to29mins15,30to34mins15,35to39mins15,40to44mins15,totPop20,5to9mins20,...,40to44mins20,state_x,county,5to14mins15,15to24mins15,25to34mins15,35to44mins15,5to14mins20,15to24mins20,25to34mins20,35to44mins20
0,214766,13036,18468,16326,12738,4724,9040,1668,2693,213505,12039,...,2004,25,001,14.668989,13.532868,6.408836,2.030582,13.207185,13.939720,7.258378,1.834617
1,129288,10867,10791,9684,7267,2898,5001,960,1371,125927,10436,...,1528,25,003,16.751748,13.111039,6.109616,1.802952,17.911171,12.562040,6.021743,2.185393
2,503681,18567,26515,28817,28203,13551,30313,7113,11100,518597,20969,...,11378,25,023,8.950506,11.320657,8.708687,3.615979,9.526280,10.866048,8.247637,3.870250
3,687721,24462,31915,36343,36082,17124,49397,12074,19240,703740,20946,...,19822,25,021,8.197656,10.531160,9.672672,4.553300,7.154631,9.798221,9.050644,4.599142
4,10556,1615,1846,1029,255,33,79,0,11,11212,1363,...,253,25,019,32.787041,12.163698,1.061008,0.104206,28.157331,12.959329,0.651088,2.301106


I choose only needed columns to reduce dataframe size

In [ ]:

commute = commute[['county','state_x', 'totPop15', 'totPop20','5to14mins15', '15to24mins15', '25to34mins15', '35to44mins15', '5to14mins20', '15to24mins20', '25to34mins20', '35to44mins20']]

In [ ]:
commute.head()

,county,state_x,totPop15,totPop20,5to14mins15,15to24mins15,25to34mins15,35to44mins15,5to14mins20,15to24mins20,25to34mins20,35to44mins20
0,001,25,214766,213505,14.668989,13.532868,6.408836,2.030582,13.207185,13.939720,7.258378,1.834617
1,003,25,129288,125927,16.751748,13.111039,6.109616,1.802952,17.911171,12.562040,6.021743,2.185393
2,023,25,503681,518597,8.950506,11.320657,8.708687,3.615979,9.526280,10.866048,8.247637,3.870250
3,021,25,687721,703740,8.197656,10.531160,9.672672,4.553300,7.154631,9.798221,9.050644,4.599142
4,019,25,10556,11212,32.787041,12.163698,1.061008,0.104206,28.157331,12.959329,0.651088,2.301106


Import geofiles and filter to only Massachusetts (with STATEFP '25')

In [ ]:
usGeo = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_5m.zip')

In [ ]:
usGeo = usGeo.to_crs('epsg:4326')

In [ ]:
#Filter down to Massachusetts
massGeo = usGeo.loc[usGeo['STATEFP'] == '25'].reset_index(drop=True)
massGeo.head(2)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,25,003,00606928,0500000US25003,25003,Berkshire,Berkshire County,MA,Massachusetts,06,2400599906,50694129,"POLYGON ((-73.50814 42.08626, -73.47592 42.174..."
1,25,019,00606936,0500000US25019,25019,Nantucket,Nantucket County,MA,Massachusetts,06,119637319,666826424,"POLYGON ((-70.27553 41.31046, -70.26063 41.310..."


In [ ]:
alt.Chart(massGeo).mark_geoshape()

alt.Chart(...)

In [ ]:
commute['GEOID'] = commute['state_x'] + commute['county']

In [ ]:
commute.head(2)

,county,state_x,totPop15,totPop20,5to14mins15,15to24mins15,25to34mins15,35to44mins15,5to14mins20,15to24mins20,25to34mins20,35to44mins20,GEOID
0,001,25,214766,213505,14.668989,13.532868,6.408836,2.030582,13.207185,13.93972,7.258378,1.834617,25001
1,003,25,129288,125927,16.751748,13.111039,6.109616,1.802952,17.911171,12.56204,6.021743,2.185393,25003


In [ ]:
commute = massGeo.merge(commute, on='GEOID')

In [ ]:
commute.head(2)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,...,state_x,totPop15,totPop20,5to14mins15,15to24mins15,25to34mins15,35to44mins15,5to14mins20,15to24mins20,25to34mins20,35to44mins20
0,25,003,00606928,0500000US25003,25003,Berkshire,Berkshire County,MA,Massachusetts,06,2400599906,...,25,129288,125927,16.751748,13.111039,6.109616,1.802952,17.911171,12.562040,6.021743,2.185393
1,25,019,00606936,0500000US25019,25019,Nantucket,Nantucket County,MA,Massachusetts,06,119637319,...,25,10556,11212,32.787041,12.163698,1.061008,0.104206,28.157331,12.959329,0.651088,2.301106


In [ ]:
# !pip install altair-latimes
# import altair_latimes as lat
# alt.themes.register('latimes', lat.theme)
# alt.themes.enable('latimes')

In [ ]:
commuteMap = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('5to14mins15:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop15',title='2015 Population', format=','),
      alt.Tooltip('5to14mins15',title='% commuting 5-14mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 5 to 14 mins to work"], 
      "subtitle": ["2015"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)

commuteMap1 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('15to24mins15:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop15',title='2015 Population', format=','),
      alt.Tooltip('15to24mins15',title='% commuting 15-24mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 15 to 24 mins to work"], 
      "subtitle": ["2015"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)

commuteMap2 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('25to34mins15:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop15',title='2015 Population', format=','),
      alt.Tooltip('25to34mins15',title='% commuting 25-34mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 25 to 34 mins to work"], 
      "subtitle": ["2015"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)

commuteMap3 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('35to44mins15:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop15',title='2015 Population', format=','),
      alt.Tooltip('35to44mins15',title='% commuting 35-44mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 35 to 44 mins to work"], 
      "subtitle": ["2015"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)

commuteMap4 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('5to14mins20:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop20',title='2020 Population', format=','),
      alt.Tooltip('5to14mins20',title='% commuting 15-24mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 5 to 14 mins to work"], 
      "subtitle": ["2020"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)

commuteMap5 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('15to24mins20:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop20',title='2020 Population', format=','),
      alt.Tooltip('15to24mins20',title='% commuting 15-24mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 15 to 24 mins to work"], 
      "subtitle": ["2020"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)

commuteMap6 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('25to34mins20:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop20',title='2020 Population', format=','),
      alt.Tooltip('25to34mins20',title='% commuting 25-34mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 25 to 34 mins to work"], 
      "subtitle": ["2020"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)



commuteMap7 = alt.Chart(commute).mark_geoshape().encode(
    color=alt.Color('35to44mins20:Q', legend=alt.Legend(title='% of population')),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('totPop20',title='2020 Population', format=','),
      alt.Tooltip('35to44mins20',title='% commuting 35-44mins to work', format='.2f')]
).properties(
    width=550,
    height=300,
    title={
      "text": ["Mass. residents commuting 35 to 44 mins to work"], 
      "subtitle": ["2020"],
    }
).project(
    type='conicConformal', 
    parallels=[45 + 37 / 60, 47 + 3 / 60], 
    rotate=[94 + 15 / 60, 0]
)


alt.hconcat(commuteMap, commuteMap4).configure(
    concat=alt.CompositionConfig(spacing=50)
)



alt.HConcatChart(...)

# While commute time in some counties in Massachusetts has worsened, it has  improved in others over the last 10 years.

The biggest change is seen in Dukes County, where the percentage of residents who commute for 15 to 24 minutes to work daily reduced from 17.20 to 12 percent despite a slight increase in the population of the count over the last 10 years. But there is a slight increase (about 1 percent ) in the number of workers who spend between 5 to 14 minutes commuting to work in the same period.

Even though a county like Berkshire has lost over 4,500 of its population over the last 10 years, the number of workers spending between 5 and 14 minutes commuting to work has increased - from 16.75% to 17.91%.

In [ ]:
alt.hconcat(commuteMap1, commuteMap5).configure(
    concat=alt.CompositionConfig(spacing=50)
)


alt.HConcatChart(...)

In [ ]:
alt.hconcat(commuteMap2, commuteMap6).configure(
    concat=alt.CompositionConfig(spacing=50)
)


alt.HConcatChart(...)

There have been slight changes in the number of workers in the state who commute to work for 25 to 34 minutes int the state. In many of the counties, the rate reduced while in sickle-shaped county of Barnstable on the east coast which as seen a small decline in population, the number of workers who commute in this time range has increased.

In [ ]:
alt.hconcat(commuteMap3, commuteMap7).configure(
    concat=alt.CompositionConfig(spacing=50)
)

alt.HConcatChart(...)